# Desenvolvimento de codigo
Apos o recebmento dos arquivos CONLL, foi feito a separação dos arquivos em suas determinadas revistas. Apos esse processo foi iniciado o processo de codificação de uma atraves da plataforma JUPYTER para extrair as informações em arquivos ais facilmentes editaveis.
Para isso forom utiizadas as segintes bibliotecas:

In [63]:
import numpy as np #Biblioteca de suporte de arrays e matriz multidimensionais
import pandas as pd #Biblioteca de manipulação e análise de dado
import os #Esse modulo prove alguns metodos para atuar com o sistema operacional, nesse projeto utilizado para gerenciamento de diretorios
import sys #Esse modulo prove acesso à algumas variaveis de sistema, nesse projeto utilizado para gerenciamento de diretorios
import matplotlib.pyplot as plt #Biblioteca padrão pala plotagem de graficos
import seaborn as sns #Biblioteca para melhoria de interface dos graficos
from pandas.io.json import json_normalize

# Desenvolvimento de funções.
Em seguida foi desenvolvido o codigo necessario para a transformação dos arquivos CONLL em arquivos com dados analisaeis, seguindo as orientções de requisitos.
## getListCount
Essa função recebe uma matriz de dados(df) e retorna uma lista com a contagem de inicidencia de uma coluna especifica(title).
Utilizada para fazer a contagem dos frames

In [64]:
def getListCount(df,title):
    listOfterms = df[title].value_counts()
    return listOfterms

# returnHeaderAndContent
Essa função recebe uma matriz de dados (df) retorna duas matrizes, uma com todos os dados referentes a primeira sentença  e outro com o restante do conteudo da matriz
Essa função é executada duas vezes, uma vez para separar o titulo da revista e o resumo todo e em seguida o titulo do resumo e o conteudo

In [65]:
def returnHeaderAndContent(df):
    for index, row in df.iterrows():
        if(index != row['ID']-1):
            index_title = index
            break
    temp = index_title
    count = temp
    while temp == count:
        if(df.iloc[temp-1]['FORM'] != df.iloc[index_title-1]['FORM']):
            break
        count = count + index_title
        temp  = temp + index_title
    return  df.iloc[:temp-index_title].reset_index(drop=True), df.iloc[temp-index_title:].reset_index(drop=True)

## getPlot
Essa função recebe uma matriz de dados(df), um nome de uma coluna(label),o nome da pasta atual(directory) 
e o nome do arquivo a ser salvo(filename).
Ele gera um grafico dos dados da matriz e salva esse grafico em um arquivo na pasta e com o nome especificados.

In [81]:
def getPlot(df,label,directory,fileName):
    sns.set(style="whitegrid")
    fig = plt.figure(figsize=(23, 10))
    fig.subplots_adjust(bottom=0.4)
    my_path = "..//exportsOfAbstracts//Journals//"+directory
    plt.xticks(rotation=70)
    try:
        ax = sns.barplot(x="index", y=label, data=df)
        my_path = "..//exportsOfAbstracts//Journals//"+directory
        fig.savefig(os.path.join(my_path, fileName))
    except:
        print("")
        
    plt.close(fig)

## exportToExcel
Essa função recebe uma matriz de dados(df), o nome da pasta atual(dirName) o nome do journal (name) e uma identificação da
parte do resumo q esta sendo salva(pos).
Gera um arquivo excel dessa matriz na pasta e com o nome especificados.

In [67]:
def exportToExcel(df,dirName,name,pos):
    allFileName = "..//exportsOfAbstracts//Journals//"+dirName+"//"+name+pos+".xls"
    try:
        makeDir = "..//exportsOfAbstracts//Journals//"+dirName
        os.makedirs( makeDir );
    except:
        print("-")
    df.to_excel(allFileName, index = None, header=True)

# 

In [68]:
from wordcloud import WordCloud
def getCloudWord(df,filen):
    dic ={}
    for index, row in df.iterrows():
        dic.update({row["index"]:row["PRED"]})
    wc = WordCloud().generate_from_frequencies(dic)
    sns.set(style="whitegrid")
    fig = plt.figure(figsize=(23, 10))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.savefig(filen)

# Processo de extração dos dados
## Listagem e criação de pastas
Utilizando a pasta do arquivo atual como base, inicialmente são listados todos os arquivos dessa pasta e adicionado na variavel directories.

In [69]:
path = "."
directories = os.listdir( path )
dirName = "..//exportsOfAbstracts//Journals"

# Create target Directory if don't exist
if not os.path.exists(dirName):
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ")
else:    
    print("Directory " , dirName ,  " already exists")

('Directory ', '..//exportsOfAbstracts//Journals', ' already exists')


## Remoção de pastas não utilizadas
São retirados os arquivos e diretorios que não são relevantes para a analise.

In [70]:
try:
    for directory in directories:
        if(directory.startswith(".")):
            directories.remove(directory)
        if(directory.endswith(".ipynb")):
            directories.remove(directory)
except:
    print("-")

In [71]:
def  getData(dt):
    dfa = pd.DataFrame(columns=["ns","ne","token","target_name"])
    ns = 0
    for index, row in dt.iterrows():
        ne=1
        for token in row["tokens"]:
            dff = pd.DataFrame(columns=["ns","ne","token"],data=[[ns,ne,token]])
            dfa = dfa.append(dff).reset_index(drop=True)
            ne  = ne+1
        for frame in row["frames"]:
            ne    = json_normalize(json_normalize(frame)["target.spans"][0])["end"]
            text   = json_normalize(json_normalize(frame)["target.spans"][0])["text"]
            target = json_normalize(frame)["target.name"][0]
            dfa.loc[(dfa['ns'] == ns) & (dfa['ne'] == ne[0]) , 'target_name'] = target
        ns = ns+1
    return dfa

## Extração de dados

É criado um laço de repetição passando por todos os diretorios do diretorio base e listado todos os arquivos desse diretorio que possuem a extensão CONLL.
    Dessa nova lista tambem é criado um novo laço de repetição para cada arquivo gera uma matriz que possue todos os dados do arquivo CONLL utilizando a função returnHeaderAndContent é possivel separar o titulo do journal do conteudo do resumo. Apos isso utilizando novamente a função returnHeaderAndContent é possivel separar o titulo do resumo do seu conteudo e separalos em matrizes diferentes.
    Utilizando essas matrizes recem criadas é utilizada a função getListCount para obter a contagem dos frames e colocado em quatro variaveis representando uma lista de frames do arquivo todo, uma representando apenas os frames do titulo do journal, uma representando a do titulo do resumo e a ultima do conteudo do resumo.
    Atravez da matriz de titulo do journal é retirado o nome do Journal e transmado em uma variavel e em seguida utilizando as funções exportToExcel com cada uma das listagens é criado um arquivo excel com a incidendia de frames. Apos essa extração é utiliada a função getPlot para cada listagem tm para gerar os graficos.
    Apos a conclusão das extrações essa repetiçã passa para o proximo arquivo da listagem de arquivos CONLL. Ao final dessa listagem ele retorna ao primeiro laço de repetição que passa para o proximo diretorio onde repete o processo até finalizar os arquivos e encerrar a extração.


In [82]:

for directory in directories:
    columns = ['ID', 'FORM' ,'LEMMA' ,'PLEMMA' ,'POS' ,'PPOS' ,
       'FEAT' ,'PFEAT' ,'HEAD' ,'PHEAD' ,'DEPREL' ,'PDEPREL','FILLPRED','PRED','APRED']
    listOfFiles = []
    folderPath = ".//{}".format(directory)
    files = os.listdir( folderPath )
    for fl in files:
        if(fl.endswith('.json')):
            listOfFiles.append("{}//{}".format(folderPath,fl))
    for i in listOfFiles:
        df_all = getData(pd.read_json(i, lines=True))
        df_title = df_all[df_all["ns"]==0]
        df_head =  df_all[df_all["ns"]==1]
        df_content = df_all[df_all["ns"]>1]
        list_all = getListCount(df_all,'target_name')[1:].to_frame().reset_index()
        list_title = getListCount(df_title,'target_name')[1:].to_frame().reset_index()
        list_head  = getListCount(df_head,'target_name')[1:].to_frame().reset_index()
        list_content = getListCount(df_content,'target_name')[1:].to_frame().reset_index()
        max_id = df_head["token"].max()
        if(max_id > 10):
            max_id = 10
        new_name =""
        for s in df_head.iloc[0:max_id]["token"]:
            new_name = new_name+"_"+s[0].upper()+s[1:]
        new_name = ''.join(e for e in new_name if e.isalnum())
        exportToExcel(list_all,directory,new_name,"_all")
        getPlot(list_all,"target_name",directory,new_name+"_all.png")
        exportToExcel(list_title,directory,new_name,"_title")
        getPlot(list_title,"target_name",directory,new_name+"_title.png")
        exportToExcel(list_head,directory,new_name,"_head")
        getPlot(list_head,"target_name",directory,new_name+"_head.png")        
        exportToExcel(list_content,directory,new_name,"_content")
        getPlot(list_content,"target_name",directory,new_name+"_content.png")

-

-
-
-
-

-
-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-

-
-
-

-

-
-
-

-

-
-
-

-

-
-
-

-
-
-
-

-

-
-
-

-

-
-
-

-

-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-

-
-
-

-

-
-
-

-
-
-
-

-

-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-

-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-

-
-
-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-

-

-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-
-
-
-

-

-
-
-

-
-
-
-

-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-

-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
